In [1]:
import os
# move cwd to /path/to/ROCCO
if os.getcwd().split('/')[-1] == 'demo_files':
    os.chdir('..')

# Differential Accessibility Testing with ROCCO
In this notebook, we show how ROCCO can be used in differential accessibility studies. Note, the purpose of this document is not to identify biologically meaningful results, but to demonstrate a template for differential analysis.

We compare tissue samples from two groups: `right` and `left` heart ventricle tissue samples, respectively.

Each sample can be downloaded from ENCODE at
```
https://www.encodeproject.org/files/[name]/@@download/[name]
```
where `[name]` is replaced by an accession label in the first column of the table below
e.g.,  `ENCFF670QFL`.

## Input and Preprocessing

### Sample Metadata

We collect the following metadata in a tab-separated file, `heart_da_sampdata.txt`, that is conformable as `coldata` for DESeq2.

There are $K_1=9$ `right` samples and $K_2=10$ `left` samples.
```
name	group
ENCFF056EAW	right
ENCFF352HQI	right
ENCFF580MBF	right
ENCFF331WZQ	right
ENCFF728DUT	right
ENCFF219SJO	right
ENCFF709NIR	right
ENCFF836NAJ	right
ENCFF658MBG	right
ENCFF670QFL	left
ENCFF456PVX	left
ENCFF310VHO	left
ENCFF097EXD	left
ENCFF804KIW	left
ENCFF420QMN	left
ENCFF627RHW	left
ENCFF771XJN	left
ENCFF440WVI	left
ENCFF704VYQ	left
```

### Generating signal tracks from BAM files
With the BAM files for each sample available in the current working directory `/path/to/ROCCO`, we run

```
python3 prep_bams.py --multi
```

to create the necessary signal tracks for construction of $\mathbf{S}_{chr}$, the input matrix for ROCCO.

Output wig files are divided by chromosome into directories `tracks_chr{}`.

## Running ROCCO

We run ROCCO on each group independently to produce two peak sets $P_{R}$ (right) and $P_{L}$ (left). We then merge these two peak sets to produce a final set $P_{R+L}$ to use for differential analysis. Splitting the groups in this way is optional.

We use the `--identifiers` parameter of ROCCO to specify a file containing tags/identifiers for each group in our sample. First, we generate the `--identifiers` files for each group from the sample metadata file:

In [2]:
!cat demo_files/heart_da_sampdata.txt | grep right | cut -f1 > right.txt
!cat demo_files/heart_da_sampdata.txt | grep left | cut -f1 > left.txt

We store the output files for each group in a separate directory using parameter `--outdir` and combine the resulting `ROCCO_out` bed files into a single group-specific bed file.

For a lightweight demo, we restrict the experiment to three chromosomes, `chr18, chr19, chr20`:

**da_demo_params.csv**
```
chromosome,input_path,budget,gamma,tau,c1,c2,c3
chr18,tracks_chr18,NULL,NULL,NULL,NULL,NULL,NULL
chr19,tracks_chr19,NULL,NULL,NULL,NULL,NULL,NULL
chr20,tracks_chr20,NULL,NULL,NULL,NULL,NULL,NULL
```

Since each entry is NULL, default parameters will be used for each chromosome.

In [3]:

!python3 ROCCO.py --outdir right --identifiers right.txt --combine right.bed -p demo_files/da_demo_params.csv --multi

cmd: python3 /work/users/n/h/nolanh/ROCCO/ROCCO_chrom.py --chrom chr18 --wig_path tracks_chr18 --budget 0.035 --gamma 1.0 --tau 0.0 --c1 1.0 --c2 1.0 --c3 1.0 --solver ECOS --bed_format 3 --outdir right --rr_iter 50
retval: 0

cmd: python3 /work/users/n/h/nolanh/ROCCO/ROCCO_chrom.py --chrom chr19 --wig_path tracks_chr19 --budget 0.035 --gamma 1.0 --tau 0.0 --c1 1.0 --c2 1.0 --c3 1.0 --solver ECOS --bed_format 3 --outdir right --rr_iter 50
retval: 0

cmd: python3 /work/users/n/h/nolanh/ROCCO/ROCCO_chrom.py --chrom chr20 --wig_path tracks_chr20 --budget 0.035 --gamma 1.0 --tau 0.0 --c1 1.0 --c2 1.0 --c3 1.0 --solver ECOS --bed_format 3 --outdir right --rr_iter 50
retval: 0

combining output files --> right.bed


In [4]:
!python3 ROCCO.py --outdir left --identifiers left.txt --combine left.bed  -p demo_files/da_demo_params.csv --multi

cmd: python3 /work/users/n/h/nolanh/ROCCO/ROCCO_chrom.py --chrom chr18 --wig_path tracks_chr18 --budget 0.035 --gamma 1.0 --tau 0.0 --c1 1.0 --c2 1.0 --c3 1.0 --solver ECOS --bed_format 3 --outdir left --rr_iter 50
retval: 0

cmd: python3 /work/users/n/h/nolanh/ROCCO/ROCCO_chrom.py --chrom chr19 --wig_path tracks_chr19 --budget 0.035 --gamma 1.0 --tau 0.0 --c1 1.0 --c2 1.0 --c3 1.0 --solver ECOS --bed_format 3 --outdir left --rr_iter 50
retval: 0

cmd: python3 /work/users/n/h/nolanh/ROCCO/ROCCO_chrom.py --chrom chr20 --wig_path tracks_chr20 --budget 0.035 --gamma 1.0 --tau 0.0 --c1 1.0 --c2 1.0 --c3 1.0 --solver ECOS --bed_format 3 --outdir left --rr_iter 50
retval: 0

combining output files --> left.bed


## Differential Analysis
Now that we have peak files for both groups, we prepare input for the differential accessibility analysis

### Merge bed files from each group


In [5]:
!cat right.bed left.bed | bedtools sort | bedtools merge > sorted_merged_right_left.bed

### Create the count matrix
`count_matrix.py` produces a sample-by-peak matrix with each element equal to the number of reads present in the respective sample alignment (row) in the peak region (column). It will sort the samples (rows) in the same order given in the sample metadata file `-m/--metadata`

In [6]:
!python3 count_matrix.py -i sorted_merged_right_left.bed -m demo_files/heart_da_sampdata.txt -o right_left.tsv

In [7]:
import pandas as pd
count_df = pd.read_csv('right_left.tsv',sep="\t",index_col='name')
count_df

name,chr18_12900_13050,chr18_75900_75950,chr18_78400_78600,chr18_85850_85900,chr18_95650_95950,chr18_96400_96500,chr18_106450_111200,chr18_111300_112600,chr18_112800_112950,chr18_158000_159600,...,chr20_64262200_64262250,chr20_64262500_64262550,chr20_64262850_64263050,chr20_64263350_64263600,chr20_64265300_64265750,chr20_64267100_64267300,chr20_64267450_64267550,chr20_64268250_64268500,chr20_64285600_64286450,chr20_64327500_64327900
ENCFF056EAW.bam,54,7,29,12,35,13,1898,401,14,3037,...,17,22,57,49,112,67,18,173,649,632
ENCFF352HQI.bam,38,21,42,16,28,23,2152,325,26,1901,...,23,35,58,61,113,97,27,225,677,421
ENCFF580MBF.bam,13,4,2,4,7,5,778,147,5,742,...,7,8,17,9,14,11,3,33,88,190
ENCFF331WZQ.bam,29,12,32,16,45,12,2215,185,11,1257,...,32,48,46,43,90,81,22,123,523,218
ENCFF728DUT.bam,46,13,26,16,37,7,2863,318,15,1268,...,20,33,38,44,64,70,23,76,240,169
ENCFF219SJO.bam,27,19,18,15,39,14,1662,293,22,1779,...,27,46,58,47,88,107,19,175,377,313
ENCFF709NIR.bam,24,4,14,9,25,15,1874,212,13,1142,...,27,36,37,41,74,83,17,95,434,111
ENCFF836NAJ.bam,7,5,4,4,4,1,590,89,5,954,...,4,9,13,13,28,29,5,49,126,144
ENCFF658MBG.bam,85,10,35,20,40,12,3640,342,14,2011,...,21,27,69,54,76,60,22,90,885,376
ENCFF670QFL.bam,15,5,24,8,22,15,1390,171,4,697,...,6,12,21,22,73,61,8,24,200,102


Sample Metadata (coldata):

In [8]:
samp_df = pd.read_csv('demo_files/heart_da_sampdata.txt',sep="\t",index_col='name')
samp_df

,group
name,
ENCFF056EAW.bam,right
ENCFF352HQI.bam,right
ENCFF580MBF.bam,right
ENCFF331WZQ.bam,right
ENCFF728DUT.bam,right
ENCFF219SJO.bam,right
ENCFF709NIR.bam,right
ENCFF836NAJ.bam,right
ENCFF658MBG.bam,right


### Running DESeq2 

We use the PyDESeq2 package (`pip install pydeseq2`) to perform the differential analysis.

In [9]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
dds = DeseqDataSet(counts=count_df, clinical=samp_df, design_factors="group", n_cpus=8)
dds.deseq2()

Fitting size factors...
... done in 0.02 seconds.

Fitting dispersions...
... done in 6.13 seconds.

Fitting dispersion trend curve...
... done in 8.78 seconds.

Fitting MAP dispersions...
... done in 6.37 seconds.

Fitting LFCs...
... done in 3.50 seconds.

Refitting 4 outliers.

Fitting dispersions...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.



In [10]:
stat_res = DeseqStats(dds, n_cpus=8)
stat_res.summary()

Running Wald tests...
... done in 2.17 seconds.

Log2 fold change & Wald test p-value: group right vs left


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
name,,,,,,
chr18_12900_13050,19.750014,0.277859,0.266202,1.043788,0.296584,0.735510
chr18_75900_75950,6.376229,0.002266,0.335083,0.006763,0.994604,NaN
chr18_78400_78600,18.363631,-0.726265,0.307226,-2.363948,0.018081,0.356926
chr18_85850_85900,7.207599,0.042162,0.331515,0.127180,0.898798,NaN
chr18_95650_95950,24.546939,-0.695299,0.296028,-2.348761,0.018836,0.363335
...,...,...,...,...,...,...
chr20_64267100_64267300,45.348227,-0.273221,0.202428,-1.349720,0.177106,0.645529
chr20_64267450_64267550,9.872817,0.056046,0.265466,0.211125,0.832790,NaN
chr20_64268250_64268500,73.247131,-0.132047,0.251459,-0.525123,0.599498,0.877299


#### Differentially Accessible Peaks
Using an FDR-corrected $p$-value threshold $\alpha=.05$, we find 42 differentially accessible peaks in `chr18-20` spanning roughly 15kb.

In [11]:
res_df = stat_res.results_df
da_peaks = res_df[res_df['padj'] < .05]
da_peaks

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
name,,,,,,
chr18_9505950_9506700,82.367298,-0.719543,0.162453,-4.429242,9.456485e-06,0.016018
chr18_12482050_12482300,30.580929,-0.808117,0.207305,-3.898204,9.690895e-05,0.048145
chr18_22113200_22113850,163.898494,-0.680514,0.174491,-3.899985,9.619853e-05,0.048145
chr18_23542050_23542250,19.606838,-0.935027,0.237387,-3.938823,8.188240e-05,0.046351
chr18_27619900_27620350,144.956233,-0.765002,0.172837,-4.426138,9.593506e-06,0.016018
chr18_29713350_29713550,19.732844,-0.906841,0.227723,-3.982205,6.827885e-05,0.045928
chr18_30679700_30679950,29.992442,-1.116671,0.252812,-4.417002,1.000791e-05,0.016018
chr18_31205000_31205100,22.222314,-0.841693,0.213436,-3.943537,8.028845e-05,0.046351
chr18_31213600_31213950,37.860407,-0.879928,0.227208,-3.872796,1.075940e-04,0.049204
